In [ ]:
# Auto-reload for development
%load_ext autoreload
%autoreload 2

# Essential imports
import lumnisai
from lumnisai import AsyncClient, ApiProvider
from dotenv import load_dotenv
from typing import Optional, Dict, Any
import asyncio
from IPython.display import Markdown, display
import json
from pydantic import BaseModel, Field
import os

# Load environment variables
load_dotenv()

def display_markdown(text: str) -> None:
    """Helper to display markdown in notebooks"""
    display(Markdown(text))


In [ ]:
class LumnisSetup:
    """Helper class for streamlined LumnisAI setup"""
    
    def __init__(self, client: Optional[AsyncClient] = None):
        self.client = client or AsyncClient()
        self._setup_complete = False
    
    async def quick_setup(
        self,
        user_email: str,
        first_name: str = "Developer",
        last_name: str = "User",
        apps_to_enable: Optional[list[str]] = None,
        model_preferences: Optional[Dict[str, Dict[str, str]]] = None,
        api_keys: Optional[Dict[str, str]] = None
    ) -> Dict[str, Any]:
        """Complete setup in one function call with smart defaults"""
        
        print("🚀 Starting LumnisAI setup...")
        
        # Set defaults
        apps_to_enable = apps_to_enable or ["github", "gmail"]
        api_keys = api_keys or {
            "OPENAI_API_KEY": os.getenv("OPENAI_API_KEY"),
            "EXA_API_KEY": os.getenv("EXA_API_KEY")
        }
        
        setup_results = {
            "user": None,
            "api_keys": [],
            "apps": [],
            "connections": [],
            "model_preferences": None,
            "errors": []
        }
        
        try:
            # 1. Setup user
            print("👤 Setting up user...")
            user = await self._ensure_user(user_email, first_name, last_name)
            setup_results["user"] = user
            print(f"   ✅ User ready: {user_email}")
            
            # 2. Setup API keys
            print("🔑 Adding API keys...")
            for key_name, key_value in api_keys.items():
                if key_value:
                    try:
                        provider = getattr(ApiProvider, key_name)
                        await self.client.add_api_key(provider, key_value)
                        setup_results["api_keys"].append(key_name)
                        print(f"   ✅ {key_name} added")
                    except Exception as e:
                        setup_results["errors"].append(f"API key {key_name}: {str(e)}")
                        print(f"   ⚠️  {key_name} failed: {str(e)}")
            
            # 3. Setup model preferences
            if model_preferences:
                print("🤖 Updating model preferences...")
                prefs = await self.client.update_model_preferences(model_preferences)
                setup_results["model_preferences"] = prefs
                print("   ✅ Model preferences updated")
            
            # 4. Enable apps
            print("🔌 Enabling apps...")
            for app in apps_to_enable:
                try:
                    enabled = await self._ensure_app_enabled(app)
                    if enabled:
                        setup_results["apps"].append(app)
                        print(f"   ✅ {app} enabled")
                except Exception as e:
                    setup_results["errors"].append(f"App {app}: {str(e)}")
                    print(f"   ⚠️  {app} failed: {str(e)}")
            
            # 5. Check connections
            print("🔗 Checking connections...")
            connections = await self.client.list_connections(user_email)
            setup_results["connections"] = connections
            print(f"   ✅ {len(connections.connections)} connections found")
            
            self._setup_complete = True
            print("\\n🎉 Setup complete!")
            
        except Exception as e:
            setup_results["errors"].append(f"Setup failed: {str(e)}")
            print(f"\\n❌ Setup failed: {str(e)}")
        
        return setup_results
    
    async def _ensure_user(self, email: str, first_name: str, last_name: str):
        """Ensure user exists, create if needed"""
        try:
            users = await self.client.list_users(page_size=50)
            existing_user = next((u for u in users.users if u.email == email), None)
            
            if existing_user:
                return existing_user
            else:
                return await self.client.create_user(
                    email=email,
                    first_name=first_name,
                    last_name=last_name
                )
        except Exception as e:
            raise Exception(f"User setup failed: {str(e)}")
    
    async def _ensure_app_enabled(self, app_name: str) -> bool:
        """Ensure app is enabled"""
        try:
            is_enabled = await self.client.is_app_enabled(app_name)
            if not is_enabled:
                await self.client.set_app_enabled(app_name, enabled=True)
            return True
        except Exception as e:
            raise Exception(f"App enable failed: {str(e)}")
    
    async def get_connection_url(self, user_email: str, app_name: str) -> Optional[str]:
        """Get connection URL for app if not already connected"""
        try:
            status = await self.client.get_connection_status(user_id=user_email, app_name=app_name)
            if status.status != "active":
                connection = await self.client.initiate_connection(user_id=user_email, app_name=app_name)
                return connection.url if hasattr(connection, 'url') else str(connection)
            return None
        except Exception as e:
            print(f"⚠️  Connection check failed for {app_name}: {str(e)}")
            return None
    
    @property
    def is_ready(self) -> bool:
        """Check if setup is complete"""
        return self._setup_complete

# Create global setup instance
setup = LumnisSetup()


In [ ]:
# Define your user email
USER_EMAIL = "your-email@example.com"  # Replace with your email

# Complete setup in one line!
results = await setup.quick_setup(
    user_email=USER_EMAIL,
    first_name="Developer",
    last_name="User"
)

# Display results
print("\\n📊 Setup Summary:")
print(f"   User: {results['user'].email if results['user'] else 'Failed'}")
print(f"   API Keys: {len(results['api_keys'])} added")
print(f"   Apps: {len(results['apps'])} enabled")
print(f"   Connections: {len(results['connections'].connections if results['connections'] else [])} active")
if results['errors']:
    print(f"   Errors: {len(results['errors'])}")
    for error in results['errors']:
        print(f"     - {error}")


In [ ]:
# Create a user-scoped client for cleaner API calls
client = setup.client.for_user(USER_EMAIL)

# Simple query with progress tracking
print("🤖 Making AI request...")
response = await client.invoke(
    "What's the weather like in Tokyo today?",
    show_progress=True
)

display_markdown(response.output_text)
